In [ ]:
import os
import sys

sys.path.append("../")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from utils import count_stage

from natsort import natsorted

In [ ]:
stages = []
HR = []
step = []
for i in range(31):
    PATH = os.path.join("../data/padding", f"c{i+1}_data.csv")
    data = pd.read_csv(PATH)
    data = data.iloc[:,1:]
    stages.append(data["psg_status"].to_numpy())
    HR.append(data["heart_rate"].to_numpy())
    step.append(data["step_count"].to_numpy())

In [ ]:
subj_stage = count_stage(stages)

In [ ]:
subj_stage/30

In [ ]:
plt.figure(figsize=(5, 5))

index = ["Wake", "N1", "N2", "N3", "REM"]

plt.bar(index, subj_stage.T.sum(axis=1))
plt.title("Total Sleep Stage", size=18)
plt.xlabel("Sleep Stage")
plt.ylabel("Number of sample (sec)")
plt.show()

In [ ]:
fig, ax = plt.subplots(4, 8, figsize=(40, 20))

j = 0
i = 0

for subj in range(len(subj_stage)):
    
    if j % 8 == 0 and j != 0:
        i += 1
        j = 0
    
    index = ["Wake", "N1", "N2", "N3", "REM"]
    ax[i][j].bar(index, subj_stage[subj])
    ax[i][j].set_title(f"Subject {subj+1}", size=18)
    j+=1
    
plt.show()

In [ ]:
fig, ax = plt.subplots(8, 4, figsize=(40, 40))

j = 0
i = 0

for subj in range(len(HR)):
    
    if j % 4 == 0 and j != 0:
        i += 1
        j = 0
    
    ax[i][j].plot(HR[subj])
    ax[i][j].set_title(f"Subject {subj+1}", size=18)
    j+=1
    
plt.show()

In [ ]:
fig, ax = plt.subplots(8, 4, figsize=(40, 40))

j = 0
i = 0

for subj in range(len(step)):
    
    if j % 4 == 0 and j != 0:
        i += 1
        j = 0
    
    ax[i][j].plot(step[subj])
    ax[i][j].set_title(f"Subject {subj+1}", size=18)
    j+=1
    
plt.show()

In [ ]:
subj_stage.sum(axis=1)

In [ ]:
PATH = os.path.join("../data", f"c1_data.csv")
data = pd.read_csv(PATH)
data = data

In [ ]:
data.iloc[:,7]

In [ ]:
def find_label_num(BASE):

    if BASE == "../data":
        for i in range(31):
            
            print(f"Subject {i}")
            PATH = os.path.join(BASE, f"c{i+1}_data.csv")
            data = pd.read_csv(PATH)
            stage = data.iloc[:,7]
            count=0
            before_stage = 0
            start_index = 0

            for i, current_stage in enumerate(stage):

                if current_stage == before_stage:
                    count+=1

                else: # 바뀔 경우
                    if count % 30 != 0:
                        print(f"{before_stage} : {count} // Need to fix {30 - count%30}")
                        print("(start, finish): ",start_index, i-1)
                        print("---------------")
                    count=1
                    start_index = i
                    before_stage = current_stage
            print("#################################")
    else:
        client_idx = [file.split("_")[0] for file in os.listdir(BASE)]
        client_idx = natsorted(client_idx)
        for client in client_idx:
            if "c" in client:
                print(f"Subject {client}")
                PATH = os.path.join(BASE, f"{client}_data.csv")
                data = pd.read_csv(PATH)
                stage = data.iloc[:,7]
                count=0
                before_stage = 0
                start_index = 0

                for i, current_stage in enumerate(stage):

                    if current_stage == before_stage:
                        count+=1

                    else: # 바뀔 경우
                        if count % 30 != 0:
                            print(f"{before_stage} : {count} // Need to fix {30 - count%30}")
                            print("(start, finish): ",start_index, i-1)
                            print("---------------")
                        count=1
                        start_index = i
                        before_stage = current_stage
                print("#################################")
        return

In [ ]:
find_label_num("../data/padding")

In [ ]:
def pading(BASE):
    
    for subj in range(31):
        print(f"Subject {subj}")
        PATH = os.path.join(BASE, f"c{subj+1}_data.csv")
        data = pd.read_csv(PATH)
        stage = data.iloc[:,7]
        count=0
        before_stage = 0
        start_index = 0

        change_dic = {"row":[], "index":[]}

        for i, current_stage in enumerate(stage):

            if current_stage == before_stage:
                count+=1

            else: # 바뀔 경우
                need_sec = 30 - count % 30

                if need_sec != 30:
                    if need_sec == 1:
                        add_row_1 = data.iloc[i,:]
                        add_row_2 = data.iloc[i+1,:]
                        add_row = (add_row_1 + add_row_2) / 2
                        change_dic["index"].append(i)
                        change_dic["row"].append(add_row)
                        # print("Add mean row for adding 1", np.array(add_row).shape)

                    elif need_sec == 29:
                        change_dic["index"].append(i)
                        change_dic["row"].append(0)
                        # print("Drop the row")


                    else:
                        row_set = []
                        change_dic["index"].append(i)
                        for i in range(need_sec):
                            add_row = data.iloc[start_index+i,:]
                            row_set.append(add_row)
                        change_dic["row"].append(row_set)
                        # print("Add row", need_sec, np.array(row_set).shape)

                count=1
                start_index = i
                before_stage = current_stage
        
        for index, row in zip(change_dic["index"], change_dic["row"]):
            change_idx = 0
            print(len(data))

            if type(row) is int:
                data.drop(data[data["Unnamed: 0"]==index].index, axis=0, inplace=True)
                print("Drop")
                change_idx -= 1

            elif len(np.array(row).shape) == 1:
                data = pd.concat([data.iloc[:index,:], pd.DataFrame(row, columns=data.columns), data.iloc[index:,:]], ignore_index=True, axis=0)
                print("Add 1 row")
                change_idx += 1

            elif len(np.array(row).shape) > 1:
                for j in range(len(row)):
                    data = pd.concat([data.iloc[:index+j+change_idx,:], row[j], data.iloc[index+j+change_idx:,:]], ignore_index=True, axis=0)
                    change_idx += 1
                print(f"Add {len(row)}row")
            print(len(data))
        print("#################################")
        data.reset_index(inplace=True)
        data.drop("Unnamed: 0", inplace=True, axis=1)
        change_data = os.path.join(BASE, "padding", f"c{subj+1}_data_pad.csv")
        data.to_csv(change_data, index=False)

In [ ]:
pading("../data")

In [ ]:
find_label_num("../data/padding")